In [15]:
import pandas as pd
from sqlalchemy import create_engine

In [16]:
mass_shooting = pd.read_excel('Mother Jones - Mass Shootings Database, 1982 - 2019.xlsx')
mass_shooting = mass_shooting[['location', 'fatalities', 'injured', 'date', 'case', 'age_of_shooter']].copy()

mass_shooting.head()

,location,fatalities,injured,date,case,age_of_shooter
0,"Dayton, OH",9,27,2019-08-04,Dayton entertainment district shooting,24
1,"El Paso, TX",22,26,2019-08-03,El Paso Walmart mass shooting,21
2,"Gilroy, CA",3,12,2019-07-28,Gilroy garlic festival shooting,19
3,"Virginia Beach, VA",12,4,2019-05-31,Virginia Beach municipal building shooting,40
4,"Aurora, IL",5,6,2019-02-15,Harry Pratt Co. warehouse shooting,45


In [19]:
mass_shooting['city'] = mass_shooting['location'].str.split(', ').str[0]
mass_shooting['state'] = mass_shooting['location'].str.split(', ').str[1]
mass_shooting= mass_shooting.rename(columns={"city": "City", "state" : "State"})

mass_shooting.head()

,location,fatalities,injured,date,case,age_of_shooter,City,State
0,"Dayton, OH",9,27,2019-08-04,Dayton entertainment district shooting,24,Dayton,OH
1,"El Paso, TX",22,26,2019-08-03,El Paso Walmart mass shooting,21,El Paso,TX
2,"Gilroy, CA",3,12,2019-07-28,Gilroy garlic festival shooting,19,Gilroy,CA
3,"Virginia Beach, VA",12,4,2019-05-31,Virginia Beach municipal building shooting,40,Virginia Beach,VA
4,"Aurora, IL",5,6,2019-02-15,Harry Pratt Co. warehouse shooting,45,Aurora,IL


In [20]:
gun_violence = "gun_violence.csv"
gun_violence_df = pd.read_csv(gun_violence)
gun_violence_df.rename(columns={'state':'State'}, inplace=True)
gun_violence_df = gun_violence_df[['State', 'city_or_county', 'n_killed', 'n_injured', 'date', 'participant_name', 'participant_age']].copy()
gun_violence_df= gun_violence_df.rename(columns={"city_or_county": "City", "n_killed" : "fatalities", "n_injured": :"injured", "participant_age": "age_of_shooter"})
gun_violence_df.head()

SyntaxError: invalid syntax (<ipython-input-20-e534246a4d98>, line 5)

In [5]:
cities = "cities.csv"
cities_df = pd.read_csv(cities)
cities_df.head()

,State,City,Population
0,Alabama,Abernant,7599.0
1,Alabama,Alabaster,31545.0
2,Alabama,Albertville,21458.0
3,Alabama,Alexander City,14849.0
4,Alabama,Andalusia,9081.0


In [6]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [7]:
#state_abbr = pd.DataFrame(us_state_abbrev) 
states= pd.DataFrame([us_state_abbrev]).transpose().reset_index()
states.head()



,index,0
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [8]:
# Rename
state_ab= states.rename(columns={"index": "State", 0 : "abbrev"})

In [9]:
#join cities df to state_ab df to attach abbreviaitons to city populations
states_pop = pd.merge(cities_df, state_ab, on='State', how='left')
states_pop.head()


,State,City,Population,abbrev
0,Alabama,Abernant,7599.0,AL
1,Alabama,Alabaster,31545.0,AL
2,Alabama,Albertville,21458.0,AL
3,Alabama,Alexander City,14849.0,AL
4,Alabama,Andalusia,9081.0,AL


In [10]:
# Create per capita measures for gun violence
## count number of gun violence occureences in each city
### Divide by population from cities df
states_pop = states_pop.drop_duplicates(subset='State', keep='first')



In [11]:
states_pop = states_pop[['State', 'abbrev', 'Population']].copy()
states_pop.head()

,State,abbrev,Population
0,Alabama,AL,7599.0
125,Alaska,AK,301010.0
143,Arkansas,AR,197706.0
200,Arizona,AZ,1537058.0
299,California,CA,3928864.0


In [12]:
per_cap_df= pd.merge(gun_violence_df, states_pop, on='State', how='left')
per_cap_df.head()

,incident_id,date,State,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,abbrev,Population
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN,PA,1560297.0
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0,CA,3928864.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0,OH,835957.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0,CO,663862.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0,NC,809958.0


In [13]:
#

In [14]:
## Join per capita incidence to mass shootings file by city, state

## Analyse relationship between per captia violence and mass shootings